In [13]:
from openai import OpenAI

openai_client = OpenAI()

In [14]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text


In [15]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        

In [16]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


In [17]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results


In [18]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, instructions=instructions)
    return answer

In [19]:
question = 'I just found the course. Can I join now?'

results = index.search(
    question,
    num_results=5
)

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [20]:
instructions = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

In [23]:
import json
def build_prompt(question, search_results):
    search_json = json.dumps(search_results)
    return prompt_template.format(
        question=question,
        context=search_json
    )
    

In [24]:
rag('how do I install Kafka in Python?')

"To install Kafka in Python, you can use the following commands:\n\n1. For the Confluent Kafka library, run:\n   ```\n   pip install confluent-kafka\n   ```\n   or, if you prefer using Conda:\n   ```\n   conda install conda-forge::python-confluent-kafka\n   ```\n\n2. If you need to use the Kafka Python client and you're facing compatibility issues with Python 3.12.1, you should uninstall the existing kafka-python package and then install it from the GitHub repository:\n   ```\n   pip uninstall kafka-python\n   pip install git+https://github.com/dpkp/kafka-python.git\n   ```\n\nMake sure to install any additional dependencies as required for your specific use case."

In [ ]:
!uv add sentence-transformers

Resolved 149 packages in 3.04s                                       
⠇ Preparing packages... (25/29)                                                 

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')